# Yelp API - Lab



## Introduction 

Now that we've seen how the Yelp API works, it's time to put those API and SQL skills to work in order to do some basic business analysis! Taking things a step further, you'll also independently explore how to perform pagination in order to retrieve a full results set from the Yelp API!

## Objectives

You will be able to:
* Create a DB on AWS to store information from Yelp about businesses
* Create HTTP requests to get data from Yelp API
* Parse HTTP responses and insert the information into your DB
* Perform pagination to retrieve troves of data!
* Write SQL queries to answer questions about your data 

## Problem Introduction

For this lab you will analyze the yelp data for a group of businesses to learn more about an industry. You will choose a type of business (Italian Restuarants, Nail Salons, Crossfit gyms) and a location to analyze. Then you will get data from the Yelp API, store that data in a SQL Database on AWS, and write queries to answer questions about the data. 


### Process:

1. Read through the SQL questions and the API documentation to determine which pieces of information you need to pull from the Yelp API.

2. Create a DB schema with 2 tables. One for the businesses and one for the reviews.

3. Create code to:
  - Perform a search of businesses using pagination
  - Parse the API response for specific data points
  - Insert the data into your AWS DB

4. Use the functions above in a loop that will paginate over the results to retrieve all of the results. 

5. Create functions to:
  - Retrieve the reviews data of one business
  - Parse the reviews response for specific review data
  - Insert the review data into the DB

6. Using SQL, query all of the business IDs. Using the 3 Python functions you've created, run your business IDs through a loop to get the reviews for each business and insert them into your DB.

7. Write SQL queries to answer the following questions about your data.


Bonus Steps:  
- Place your helper functions in a package so that your final notebook only has the major steps listed.
- Rewrite your business search functions to be able take an argument for the type of business you are searching for.
- Add another group of businesses to your database.


 
## SQL Questions:

- Which are the 5 most reviewed businesses?
- What is the highest rating recieved in your data set and how many businesses have that rating?
- What percentage of businesses have a rating greater than or  4.5?
- What percentage of businesses have a rating less than 3?
- What is the average rating of restaurants that have a price label of one dollar sign? Two dollar signs? Three dollar signs? 


- Return the text of the reviews for the most reviewed restaurant. 
- Return the name of the business with the most recent review. 
- Find the highest rated business and return text of the most recent review. If multiple business have the same rating, select the restaurant with the most reviews. 
- Find the lowest rated business and return text of the most recent review.  If multiple business have the same rating, select the restaurant with the least reviews. 


## Part I - Set up the DB

Start by reading SQL questions above to get an understanding of the data you will need. Then, read the documentation of Yelp API to understand what data you will receive in the response.  


Now that you are familiar with the data, create your SQL queries to create the DB and the appropriate tables. 

In [71]:
import mysql.connector 

In [72]:
import config

In [73]:
cnx = mysql.connector .connect(
    host = config.host,
    user = config.user, 
    passwd = config.passwd,
    database = db_name
)
print(cnx)

In [74]:
## Connect to DB server on AWS
cursor = cnx.cursor()

In [75]:
from mysql.connector import errorcode

In [6]:
## Create new DB 
db_name = "Plumbs"

In [531]:
def create_database(cursor, database):
    try:
        cursor.execute(
            "CREATE DATABASE {} DEFAULT CHARACTER SET 'utf8'".format(database))
    except mysql.connector.Error as err:
        print("Failed creating database: {}".format(err))
        exit(1)

try:
    cursor.execute("USE {}".format(db_name))
except mysql.connector.Error as err:
    print("Database {} does not exists.".format(db_name))
    if err.errno == errorcode.ER_BAD_DB_ERROR:
        create_database(cursor, db_name)
        print("Database {} created successfully.".format(db_name))
        cnx.database = db_name
    else:
        print(err)
        exit(1)

In [18]:
# Create a table for the Businesses

db_name = 'Plumbs'

TABLES = {}
TABLES['Plumb_biz'] = (
    "CREATE TABLE Plumb_biz ("
    "  biz_id varchar(40) NOT NULL PRIMARY KEY,"
    "  biz_name varchar(100) NOT NULL,"
    "  review_count int(16) NOT NULL,"
    "  price varchar(11) NOT NULL,"
    "  rating int(7) NOT NULL"
    ") ENGINE=InnoDB")

TABLES['Plumb_Reviews'] = (
    "CREATE TABLE Plumb_Reviews ("
    "  rev_id int(71) NOT NULL PRIMARY KEY,"
    "  rev_rating int(8) NOT NULL,"
    "  name varchar(40) NOT NULL,"
    "  user_id varchar(40) NOT NULL,"
    "  review_text varchar(255) NOT NULL,"
    "  rev_date date NOT NULL,"
    "  biz_id varchar(40) NOT NULL,"
    "  FOREIGN KEY(biz_id) REFERENCES Plumb_biz (biz_id)"
    ") ENGINE=InnoDB")



In [19]:
for table_name in TABLES:
    table_description = TABLES[table_name]
    try:
        print("Creating table {}: ".format(table_name), end='')
        cursor.execute(table_description)
    except mysql.connector.Error as err:
        if err.errno == errorcode.ER_TABLE_EXISTS_ERROR:
            print("already exists.")
        else:
            print(err.msg)
    else:
        print("OK")

# cursor.close()
# cnx.close()

Creating table Plumb_biz: already exists.
Creating table Plumb_Reviews: OK


In [17]:
cursor.execute("""TRUNCATE Plumb_biz;""")


In [16]:
cursor.execute("""DROP Table Plumb_Reviews;""")

In [ ]:
# Create a table for the reviews

## Part 2: Create ETL pipeline for the business data from the API

In [10]:
# Write a function to make a call to the yelp API
import json
import requests
import pandas as pd
import time

api_key = "NwZvu1WZrjub3P2jL3S_GuZtYSkXkuf2J7zPLOrnPTalaDpzhTVFjNHJW2b2J1r0ILB-XU4qSwnEZ5JckShoShJYrOiOAw6ruSgpsbIxgKv40UuNscGhdAWW7uixXXYx"


term = 'plumber'
location = 'Bushwick NY'
url_params = {  'term': term.replace(' ', '+'),
                'location': location.replace(' ', '+'),
                'limit' : 50
             }


url = 'https://api.yelp.com/v3/businesses/search'
headers = {'Authorization': 'Bearer {}'.format(api_key)}
response = requests.get(url, headers=headers, params=url_params)



def yelp_call(url_params, api_key):
    url = 'https://api.yelp.com/v3/businesses/search'
    headers = {'Authorization': 'Bearer {}'.format(api_key)}
    response = requests.get(url, headers=headers, params=url_params)
    

    data = response.json()['businesses']
    return data






# def all_results(url_params, api_key):
#     num = response.json()['total']
#     print('{} total matches found.'.format(num))
#     cur = 0
    
#     while cur < num and cur < 1000:
#         url_params['offset'] = cur
#         results = yelp_call(url_params, api_key)
#         parsed_results = parse_results(results)
#         db_insert(plumb_list)
#         time.sleep(1) #Wait a second
#         cur += 50




# df = all_results(url_params, api_key)
# print(len(df))
# df.head()


In [11]:
results = yelp_call(url_params, api_key)
results

[{'id': 'tXtXY7r8NXPk0-MKJlrOjQ',
  'alias': 'friedman-irwin-and-son-brooklyn',
  'name': 'Friedman Irwin & Son',
  'image_url': 'https://s3-media1.fl.yelpcdn.com/bphoto/jD4myr5l9ob1aSe7rCFq1w/o.jpg',
  'is_closed': False,
  'url': 'https://www.yelp.com/biz/friedman-irwin-and-son-brooklyn?adjust_creative=RD6sXbiK7pfyZyJHpmys3w&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=RD6sXbiK7pfyZyJHpmys3w',
  'review_count': 50,
  'categories': [{'alias': 'plumbing', 'title': 'Plumbing'}],
  'rating': 4.5,
  'coordinates': {'latitude': 40.72369, 'longitude': -73.95462},
  'transactions': [],
  'location': {'address1': '139 Banker St',
   'address2': '',
   'address3': '',
   'city': 'Brooklyn',
   'zip_code': '11222',
   'country': 'US',
   'state': 'NY',
   'display_address': ['139 Banker St', 'Brooklyn, NY 11222']},
  'phone': '+17187827150',
  'display_phone': '(718) 782-7150',
  'distance': 4203.5129390794245},
 {'id': '1M5qDSvsfNogoZeXYx2icg',
  'alias': 'pipe-monkeys

In [13]:
results = yelp_call(url_params, api_key)
def parse_results_2(results):
    parsed_plumbs = []
    for result in results:
        plumb_dict = {}
        for key, value in result.items():
            if key == 'id':
                plumb_dict['biz_id'] = value
            if key == 'name':
                plumb_dict['biz_name'] = value
            if key == 'rating':
                plumb_dict['rating'] = value
            if key == 'review_count':
                plumb_dict['review_count'] = value
        parsed_plumbs.append(plumb_dict)
    return parsed_plumbs
parsed_plumbs = parse_results_2(results)
parsed_plumbs

[{'biz_id': 'tXtXY7r8NXPk0-MKJlrOjQ',
  'biz_name': 'Friedman Irwin & Son',
  'review_count': 50,
  'rating': 4.5},
 {'biz_id': '1M5qDSvsfNogoZeXYx2icg',
  'biz_name': 'Pipe Monkeys Sewer and Drain',
  'review_count': 28,
  'rating': 5.0},
 {'biz_id': 'hlMkCkl-8wmFD57rD_JEJg',
  'biz_name': 'Dov Sewer and Drain',
  'review_count': 56,
  'rating': 4.5},
 {'biz_id': 'a6TKGWUFop1T-nlEIi8aNA',
  'biz_name': "Sam's Plumbing Services",
  'review_count': 19,
  'rating': 4.5},
 {'biz_id': 'TYdz9vpLiXFF7-kbwbx1cQ',
  'biz_name': 'Parkset Plumbing',
  'review_count': 20,
  'rating': 4.5},
 {'biz_id': 'g5kfI33jirjUxeYPK6DU_w',
  'biz_name': 'Michael Donahue Plumbing & Heating',
  'review_count': 25,
  'rating': 5.0},
 {'biz_id': '3020UCNczgSJLYD1-JrDMw',
  'biz_name': 'Four Season Piping & Mechanical',
  'review_count': 24,
  'rating': 4.5},
 {'biz_id': '4PJwxWz0V6lcLjPJ-97H0A',
  'biz_name': 'Alexanders Proficient Heating & Cooling',
  'review_count': 38,
  'rating': 4.0},
 {'biz_id': 'QzpYYXgFs

In [ ]:
results = yelp_call(url_params, api_key)
def parse_results_2(results):
    parsed_plumbs = []
    for result in results:
        plumb_dict = 
        plumb_dict['biz_id']= result['id']
        plumb_dict[]
        for key, value in result.items():
            if key == 'id':
                plumb_dict['biz_id'] = value
            if key == 'name':
                plumb_dict['biz_name'] = value
            if key == 'rating':
                plumb_dict['rating'] = value
            if key == 'review_count':
                plumb_dict['review_count'] = value
        parsed_plumbs.append(plumb_dict)
    return parsed_plumbs
parsed_plumbs = parse_results_2(results)
parsed_plumbs

In [23]:
plumb_list = []
for plumb in parsed_plumbs:

    plumb_tuple = (plumb['biz_id'], plumb['biz_name'], 
                     plumb['review_count'], plumb['rating'])
    plumb_list.append(plumb_tuple)
    
plumb_list

[('tXtXY7r8NXPk0-MKJlrOjQ', 'Friedman Irwin & Son', 50, 4.5),
 ('1M5qDSvsfNogoZeXYx2icg', 'Pipe Monkeys Sewer and Drain', 28, 5.0),
 ('hlMkCkl-8wmFD57rD_JEJg', 'Dov Sewer and Drain', 56, 4.5),
 ('a6TKGWUFop1T-nlEIi8aNA', "Sam's Plumbing Services", 19, 4.5),
 ('TYdz9vpLiXFF7-kbwbx1cQ', 'Parkset Plumbing', 20, 4.5),
 ('g5kfI33jirjUxeYPK6DU_w', 'Michael Donahue Plumbing & Heating', 25, 5.0),
 ('3020UCNczgSJLYD1-JrDMw', 'Four Season Piping & Mechanical', 24, 4.5),
 ('4PJwxWz0V6lcLjPJ-97H0A',
  'Alexanders Proficient Heating & Cooling',
  38,
  4.0),
 ('QzpYYXgFsz7aCLgm1goDaA', 'The Original 718 Sewer & Drain', 172, 4.5),
 ('r-UxNIwsZH1KSnMkkHhZvQ', 'Franco Belli Plumbing & Heating & Sons', 32, 4.5),
 ('ZnMyiLNJqInJbQ1n-zpSSA', 'JPeral Piping & Heating Sewer Service', 45, 4.5),
 ('U4eItfkb9Mfa_vDa6d76Mg', 'The Pump Guys', 1, 5.0),
 ('dNdqqq7xqTzEYSDybMyS-Q', 'JCA Mechanical', 126, 4.5),
 ('han8zgVwsxnd_fHrco8P9w', 'Vuksani Plumbing & Heating', 5, 5.0),
 ('BF1e6ud2EWwENkyyNnh7FQ', 'ASAP Solu

In [25]:
# def all_results(url_params, api_key):
num = response.json()['total']
print('{} total matches found.'.format(num))
cur = 0

while cur < num:
    print(cur)
    url_params['offset'] = cur
    results = yelp_call(url_params, api_key)
    parsed_plumbs = parse_results_2(results)
    time.sleep(1) #Wait a second
    print(parsed_plumbs[-1])
    db_insert(parsed_plumbs)
    cur += 50
    

438 total matches found.
0
{'biz_id': 'nshCt83YkUseutjNzidTNg', 'biz_name': '39.99 sewer service', 'review_count': 7, 'rating': 2.0}
50
{'biz_id': 'ux-b_AJkRZ7cKUI5GouAxA', 'biz_name': 'Fred Smith Plumbing & Heating', 'review_count': 34, 'rating': 2.5}
100
{'biz_id': 'wO56AdkEr7cxMDeZUQ7AGQ', 'biz_name': 'Garofalo Master Plumbing & Heating Corporation', 'review_count': 6, 'rating': 4.5}
150
{'biz_id': 'NoulDZM3OUPR4Wd5ebJclA', 'biz_name': 'PP Cousin Plumbing Mechanical', 'review_count': 5, 'rating': 4.0}
200
{'biz_id': 'cvmgq9IWizNkGcm9eHbyIA', 'biz_name': 'SMB Plumbing and Heating', 'review_count': 3, 'rating': 3.0}
250
{'biz_id': 'MO2AxMYZ-coBAl7t0uiDLw', 'biz_name': 'Scaran', 'review_count': 20, 'rating': 4.0}
300
{'biz_id': 'i-fjCqNziJwR5NfhQn0HDQ', 'biz_name': 'J P Sewer and Water Mains', 'review_count': 3, 'rating': 2.5}
350
{'biz_id': 'tNYiixKuXb6FcL38g8MtBg', 'biz_name': 'ATG Mechanical', 'review_count': 6, 'rating': 1.5}
400
{'biz_id': '5wVXlHH3YWcR_f5BRZ9-Eg', 'biz_name': 'Gu

In [ ]:
def db_insert(parsed_results):
    plumb_list = []
    for plumb in parsed_plumbs:

        plumb_tuple = (plumb['biz_id'], plumb['biz_name'], 
                         plumb['review_count'], plumb['rating'])
        plumb_list.append(plumb_tuple)
    insert_statement = "INSERT INTO Plumb_biz (biz_id, biz_name, review_count, rating)  VALUES (%s, %s, %s, %s)"
    cursor.executemany(insert_statement, plumb_list)
    cnx.commit()

In [42]:
locale = 'enUS'
url_params = { 'local': locale,
            'limit': 3}

url = 'https://api.yelp.com/v3/businesses/{o0WJDT2-EXDUFNtmQ2wEqA}/reviews'
headers = {'Authorization': 'Bearer {}'.format(api_key)}
response = requests.get(url, headers=headers, params=url_params)

In [51]:
import json
import requests
import pandas as pd

api_key = "NwZvu1WZrjub3P2jL3S_GuZtYSkXkuf2J7zPLOrnPTalaDpzhTVFjNHJW2b2J1r0ILB-XU4qSwnEZ5JckShoShJYrOiOAw6ruSgpsbIxgKv40UuNscGhdAWW7uixXXYx"

def yelp_call_r(url_params, api_key):
    url = 'https://api.yelp.com/v3/businesses/nshCt83YkUseutjNzidTNg/reviews'
    headers = {'Authorization': 'Bearer {}'.format(api_key)}
    response = requests.get(url, headers=headers, params=url_params)
    

    data_r = response.json()['reviews']
    return data_r



In [53]:
reviews = yelp_call_r(url_params, api_key)
reviews

[{'id': 'hzTq30CWjHyeHJBLpPx-0g',
  'url': 'https://www.yelp.com/biz/39-99-sewer-service-brooklyn-3?adjust_creative=RD6sXbiK7pfyZyJHpmys3w&hrid=hzTq30CWjHyeHJBLpPx-0g&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_reviews&utm_source=RD6sXbiK7pfyZyJHpmys3w',
  'text': 'they were amazing. very professional and quick. showed up less than 3 hours after i called. were very fast and  thorough. definitely recommend them.',
  'rating': 5,
  'time_created': '2019-06-03 23:45:03',
  'user': {'id': '72fvOCOBpvLq5ZPkbx_GtQ',
   'profile_url': 'https://www.yelp.com/user_details?userid=72fvOCOBpvLq5ZPkbx_GtQ',
   'image_url': 'https://s3-media1.fl.yelpcdn.com/photo/VhgOo3FRa6Yb9y4bLUFe4g/o.jpg',
   'name': 'Danielle S.'}},
 {'id': 'wMR0sWXKej6aKVvl1L34kQ',
  'url': 'https://www.yelp.com/biz/39-99-sewer-service-brooklyn-3?adjust_creative=RD6sXbiK7pfyZyJHpmys3w&hrid=wMR0sWXKej6aKVvl1L34kQ&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_reviews&utm_source=RD6sXbiK7pfyZyJHpmys3w',
  'text': "Pl

In [66]:
reviews = yelp_call_r(url_params, api_key)
def parse_reviews_(reviews):
    parsed_reviews = []
    for review in reviews:
        rev_dict = {}
        for key, value in review.items():
            if key == 'id':
                rev_dict['rev_id'] = value
            if key == 'name':
                rev_dict['name'] = value
            if key == 'user':
                rev_dict['user'] = value
            if key == 'text':
                rev_dict['review_text'] = value
            if key == 'time_created':
                rev_dict['rev_date'] = value
        parsed_reviews.append(rev_dict)
    return parsed_reviews
par_reviews = parse_reviews_(reviews)
par_reviews




[{'rev_id': 'hzTq30CWjHyeHJBLpPx-0g',
  'review_text': 'they were amazing. very professional and quick. showed up less than 3 hours after i called. were very fast and  thorough. definitely recommend them.',
  'rev_date': '2019-06-03 23:45:03',
  'user': {'id': '72fvOCOBpvLq5ZPkbx_GtQ',
   'profile_url': 'https://www.yelp.com/user_details?userid=72fvOCOBpvLq5ZPkbx_GtQ',
   'image_url': 'https://s3-media1.fl.yelpcdn.com/photo/VhgOo3FRa6Yb9y4bLUFe4g/o.jpg',
   'name': 'Danielle S.'}},
 {'rev_id': 'wMR0sWXKej6aKVvl1L34kQ',
  'review_text': "Plumber Mike came an hour late for an appointment \nAnd he charge $140 for just diagnose \nHe said he don't have license to fix and \nLater I have called the...",
  'rev_date': '2019-10-20 11:55:29',
  'user': {'id': 'Qgq2msC6XzGsAIRhn5LDJg',
   'profile_url': 'https://www.yelp.com/user_details?userid=Qgq2msC6XzGsAIRhn5LDJg',
   'image_url': 'https://s3-media2.fl.yelpcdn.com/photo/SpZU6WKNa452Mgl1pNMe1g/o.jpg',
   'name': 'Koen Y.'}},
 {'rev_id': 'icazG

In [68]:
rev_list = []
for rev in par_reviews:

    rev_tuple = (rev['rev_id'], 
                     rev['user'], rev['review_text'], rev['rev_date'])
    rev_list.append(rev_tuple)
    
rev_list

[('hzTq30CWjHyeHJBLpPx-0g',
  {'id': '72fvOCOBpvLq5ZPkbx_GtQ',
   'profile_url': 'https://www.yelp.com/user_details?userid=72fvOCOBpvLq5ZPkbx_GtQ',
   'image_url': 'https://s3-media1.fl.yelpcdn.com/photo/VhgOo3FRa6Yb9y4bLUFe4g/o.jpg',
   'name': 'Danielle S.'},
  'they were amazing. very professional and quick. showed up less than 3 hours after i called. were very fast and  thorough. definitely recommend them.',
  '2019-06-03 23:45:03'),
 ('wMR0sWXKej6aKVvl1L34kQ',
  {'id': 'Qgq2msC6XzGsAIRhn5LDJg',
   'profile_url': 'https://www.yelp.com/user_details?userid=Qgq2msC6XzGsAIRhn5LDJg',
   'image_url': 'https://s3-media2.fl.yelpcdn.com/photo/SpZU6WKNa452Mgl1pNMe1g/o.jpg',
   'name': 'Koen Y.'},
  "Plumber Mike came an hour late for an appointment \nAnd he charge $140 for just diagnose \nHe said he don't have license to fix and \nLater I have called the...",
  '2019-10-20 11:55:29'),
 ('icazGtrRQJlk6w43icQRyQ',
  {'id': '84gtFLzvGojpvOSo0W6uRA',
   'profile_url': 'https://www.yelp.com/user_

In [24]:
url = 'https://api.yelp.com/v3/businesses/o0WJDT2-EXDUFNtmQ2wEqA/reviews'
headers = {'Authorization': 'Bearer {}'.format(api_key)}
response = requests.get(url, headers=headers, params=url_params)


def all_results_r(url_params, api_key):
    num = response.json()['total']
    print('{} total matches found.'.format(num))
    cur = 0
    
    while cur < num and cur < 1000:
        url_params['offset'] = cur
        results = yelp_call(url_params, api_key)
        parsed_results = parse_results(results)
#         db_insert(parsed)
        time.sleep(1) #Wait a second
        cur += 50

import time

# term = 'plumber'
# location = 'Bushwick NY'
# url_params = {  'term': term.replace(' ', '+'),
#                 'location': location.replace(' ', '+'),
#                 'limit' : 50
#              }
# df_2r = all_results(url_params, api_key)
# print(len(df_2r))
# df.head()
    



In [80]:
cursor.execute("""
SELECT
    biz_id
FROM
    Plumb_biz
""")
biz_rev_id = cursor.fetchall()
biz_rev_id

[('-CsL219aKfHi7iPw0YSVEQ',),
 ('-GqZkEMSA2O-7wc6nEiG4w',),
 ('-YtGo9TQjaEV7K3gPFmQuQ',),
 ('00wogt3EfiEJW2T0weSr1w',),
 ('0JES2rLBuWvsfQ249PpWbg',),
 ('0m1dRGzPNlx_JLKo6lRaTQ',),
 ('1Bq3rwwe7vPEZvUA3Q9PUw',),
 ('1Eic2ZXa4BUYZfJb65WO7g',),
 ('1M5qDSvsfNogoZeXYx2icg',),
 ('1pnxGWn5aOTV2LKjC91DAw',),
 ('1Sefmcnb_FdvQrsALgFEcQ',),
 ('2n1uXZtrN2vIY9iKku6zTw',),
 ('2rMHF-tMnF_I_PCN0MM-UQ',),
 ('2U-ekZMq7IFin7JvY-RSNQ',),
 ('2_j3xkRZXsI1_WIVJsacsg',),
 ('3020UCNczgSJLYD1-JrDMw',),
 ('37gZ1A8V-WwghnSawpEi1w',),
 ('3ecwPT6aKE1sV1F4kydJsw',),
 ('3JJdXZaaI0JOZ0kUWzAEBw',),
 ('3NfHSucKngDvz24lRnDNjA',),
 ('4-BL7dutEIm8WQNvsvPqdw',),
 ('45hg5t6uHHzVNVxx4h9Cnw',),
 ('47eRRdi0QEeHlP0BGZKtNQ',),
 ('4K7AoV3ERYL7T2allzERnA',),
 ('4oiVR8uY1VtUA06PJ5bA_Q',),
 ('4PJwxWz0V6lcLjPJ-97H0A',),
 ('4QLDHUwDRlPlfjW_KTbyDQ',),
 ('4uCT_ZMzv7jUojQgt1qzkw',),
 ('4VZoPBVtKRqt7bowq5iNzA',),
 ('4Zz720ccHd7_scM_dEFH8w',),
 ('51AwoyBg4y5pqwAw1RsUyg',),
 ('5fvwivv1vcZNOYWpgURiRg',),
 ('5T6_inkams7H4hSoWqny9w',),
 ('5UxKiYO

In [ ]:
def yelp_call_review(api_key, biz_ids):
    for i in biz_ids:
        url = ('https://api.yelp.com/v3/businesses/%s/reviews'% i)
        headers = {'Authorization': 'Bearer {}'.format(api_key)}
        response = requests.get(url, headers=headers, params=url_params)
        data = response.json()['total']
#         time.sleep(.5)
    return data

In [104]:
yelp_call_review(api_key, biz_rev_id)

KeyError: 'reviews'

In [101]:
new_biz_rev_id = biz_rev_id[0:5]
new_biz_rev_id

[('-CsL219aKfHi7iPw0YSVEQ',),
 ('-GqZkEMSA2O-7wc6nEiG4w',),
 ('-YtGo9TQjaEV7K3gPFmQuQ',),
 ('00wogt3EfiEJW2T0weSr1w',),
 ('0JES2rLBuWvsfQ249PpWbg',)]

## Part 3: Create ETL pipeline for the restaurant review data from the API

In [479]:
cnx

In [ ]:
# write a SQL query to pull back all of the business ids 
# you will need these ids to pull back the reviews for each restaurant

In [ ]:
# write a function that takes a business id 
# and makes a call to the API for reivews


In [ ]:
# Write a function to parse out the relevant information from the reviews

In [ ]:
# write a function to insert the parsed data into the reviews table

In [ ]:
# combine the functions above into a single script  

## Part 4: Write SQL queries that will answer the questions posed. 

In [ ]:
# create connection

In [ ]:
# execute SQL queries

# Extra Reference help

###  Pagination

Returning to the Yelp API, the [documentation](https://www.yelp.com/developers/documentation/v3/business_search) also provides us details regarding the API limits. These often include details about the number of requests a user is allowed to make within a specified time limit and the maximum number of results to be returned. In this case, we are told that any request has a maximum of 50 results per request and defaults to 20. Furthermore, any search will be limited to a total of 1000 results. To retrieve all 1000 of these results, we would have to page through the results piece by piece, retriving 50 at a time. Processes such as these are often refered to as pagination.

Now that you have an initial response, you can examine the contents of the json container. For example, you might start with ```response.json().keys()```. Here, you'll see a key for `'total'`, which tells you the full number of matching results given your query parameters. Write a loop (or ideally a function) which then makes successive API calls using the offset parameter to retrieve all of the results (or 5000 for a particularly large result set) for the original query. As you do this, be mindful of how you store the data. 

**Note: be mindful of the API rate limits. You can only make 5000 requests per day, and APIs can make requests too fast. Start prototyping small before running a loop that could be faulty. You can also use time.sleep(n) to add delays. For more details see https://www.yelp.com/developers/documentation/v3/rate_limiting.**

***Below is sample code that you can use to help you deal with the pagination parameter and bring all of the functions together.***


***Also, something might cause your code to break while it is running. You don't want to constantly repull the same data when this happens, so you should insert the data into the database as you call and parse it, not after you have all of the data***


In [ ]:
def yelp_call(url_params, api_key):
    url = 'https://api.yelp.com/v3/businesses/search'
    headers = {'Authorization': 'Bearer {}'.format(api_key)}
    response = requests.get(url, headers=headers, params=url_params)
    
    data = response.json()['businesses']
    return data

In [ ]:
# Your code here; use a function or loop to retrieve all the results from your original request
import time



def all_results(url_params, api_key):
    num = response.json()['total']
    print('{} total matches found.'.format(num))
    cur = 0
    
    while cur < num and cur < 1000:
        url_params['offset'] = cur
        results = yelp_call(url_params, api_key)
        parsed_results = parse_results(results)
        db_insert(parsed)
        time.sleep(1) #Wait a second
        cur += 50

term = 'pizza'
location = 'Astoria NY'
url_params = {  'term': term.replace(' ', '+'),
                'location': location.replace(' ', '+'),
                'limit' : 50
             }
df = all_results(url_params, api_key)
print(len(df))
df.head()

### Sample SQL Query 

Below is a SQL query to create a table.  Additionally here is a link to create a table with a foreign key.

http://www.mysqltutorial.org/mysql-foreign-key/

```CREATE TABLE IF NOT EXISTS tasks (
    task_id INT AUTO_INCREMENT,
    title VARCHAR(255) NOT NULL,
    start_date DATE,
    due_date DATE,
    status TINYINT NOT NULL,
    priority TINYINT NOT NULL,
    description TEXT,
    PRIMARY KEY (task_id)
)  ENGINE=INNODB;```

### Using DB:
    
For this lab, you can either store the data on one DB or put in on both of the partners DBs. If you decide to put it on one DB, you want to make sure both partners have access to it.  To do this you want to add a user to your DB.  

[how to add a new user](https://howchoo.com/g/mtm3zdq2nzv/how-to-add-a-mysql-user-and-grant-privileges)